In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout,Flatten
from keras.optimizers import SGD
from keras.activations import relu, tanh, elu
from keras.backend import clear_session
from keras.models import load_model

Using TensorFlow backend.


In [2]:
train_data_x = pd.read_pickle('normalized_fer2013.pkl')
train_data_y = pd.read_pickle('normalized_fer2013_labels.pkl').astype(int)
test_data_x = pd.read_pickle('normalized_test_fer2013.pkl')
test_data_y = pd.read_pickle('normalized_test_fer2013_labels.pkl').astype(int)

train_data_x = train_data_x.as_matrix().reshape((-1,48,48,1))
test_data_x = test_data_x.as_matrix().reshape((-1,48,48,1))

emotion = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}
positive_emotes = [3, 5]
neutral = [6]
negative_emotes = [0, 1, 2, 4]

def lump_labels(label):
    if label in negative_emotes + neutral:
        return 0
    elif label in positive_emotes:
        return 1
    else:
        return 2
    

test_data_y2 = test_data_y.apply(lump_labels)

train_data_y2 = train_data_y.apply(lump_labels)

train_data_y = train_data_y.as_matrix()
test_data_y = test_data_y.as_matrix()

train_data_y2 = train_data_y2.as_matrix()
test_data_y2 = test_data_y2.as_matrix()

train_data_y = keras.utils.to_categorical(train_data_y, num_classes=7)
test_data_y = keras.utils.to_categorical(test_data_y, num_classes=7)

train_data_y2 = keras.utils.to_categorical(train_data_y2, num_classes=2)
test_data_y2 = keras.utils.to_categorical(test_data_y2, num_classes=2)

# Training Process

In [3]:
clear_session()
model = keras.models.Sequential()

PATIENCE = 7
FILEPATH = "weights.epoch-{epoch:02d}-val_loss-{val_loss:.2f}-train_loss-{loss:.2f}.hdf5"
CSV_FILENAME = "train.log"
model.add(Dropout(0.5,input_shape=(48,48,1)))
model.add(Conv2D(8, (5,5), strides=(1,1),activation='elu',padding='valid'))
# model.add(Conv2D(16, (5,5), strides=(1,1),activation='elu',padding='valid'))
# model.add(Conv2D(32, (5,5), strides=(1,1),activation='elu',padding='valid'))

model.add(Flatten())
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                            patience=PATIENCE, verbose=0, mode='auto')
checkpoint = keras.callbacks.ModelCheckpoint(FILEPATH, monitor='val_loss', 
                    verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=0)
csv_logger = keras.callbacks.CSVLogger(CSV_FILENAME, separator=',', append=False)


In [4]:
model.fit(train_data_x, train_data_y2, validation_data=(test_data_x,test_data_y2), 
          epochs=1000, batch_size=32, callbacks=[early_stopping, checkpoint, csv_logger])

Train on 28709 samples, validate on 7178 samples
Epoch 1/1000
28709/28709 [==============================] - 13s - loss: 0.6500 - acc: 0.6369 - val_loss: 0.6150 - val_acc: 0.6651
Epoch 2/1000
28709/28709 [==============================] - 13s - loss: 0.6238 - acc: 0.6570 - val_loss: 0.6151 - val_acc: 0.6571
Epoch 3/1000
28709/28709 [==============================] - 13s - loss: 0.6204 - acc: 0.6637 - val_loss: 0.6118 - val_acc: 0.6725
Epoch 4/1000
28709/28709 [==============================] - 14s - loss: 0.6184 - acc: 0.6623 - val_loss: 0.6190 - val_acc: 0.6673
Epoch 5/1000
28709/28709 [==============================] - 14s - loss: 0.6172 - acc: 0.6646 - val_loss: 0.6107 - val_acc: 0.6669
Epoch 6/1000
28709/28709 [==============================] - 13s - loss: 0.6179 - acc: 0.6621 - val_loss: 0.6088 - val_acc: 0.6644
Epoch 7/1000
28709/28709 [==============================] - 13s - loss: 0.6180 - acc: 0.6639 - val_loss: 0.6100 - val_acc: 0.6733
Epoch 8/1000
28709/28709 [===============

KeyboardInterrupt: 